In [1]:
!pip install transformers
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.6 MB/s eta 0:00:00
TF version:  2.11.0
Hub version:  0.13.0


In [2]:
import numpy as np
import pandas as pd

# Transformers models
# You can add to these how ever you like

from transformers import RobertaTokenizer, TFRobertaModel
from transformers import BertTokenizer, TFBertModel
from transformers import XLNetTokenizer, TFXLNetModel

from tensorflow.keras.optimizers import Adam
import os
# For text preprocessing
import string
import regex as re

In [3]:
df = pd.read_csv('label.csv')

In [4]:
posneg = df[~(df['label'] == 3)]

In [5]:
posneg['sentiment'] = posneg['label'].apply(lambda x: 1 if x > 3 else 0)

<ipython-input-5-9ee1b9ce4e45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posneg['sentiment'] = posneg['label'].apply(lambda x: 1 if x > 3 else 0)


In [6]:
dataset = posneg[['TEXT', 'sentiment']]

In [7]:
dataset['sentiment'].value_counts()

1    2064
0     224
Name: sentiment, dtype: int64

In [38]:
negtrue = dataset[dataset['sentiment'] == 0]

In [8]:
neg = dataset[dataset['sentiment'] == 0]['TEXT'].values.tolist()

In [9]:
!pip install numpy requests nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 24.7 MB/s eta 0:00:00


In [10]:
!pip install torch>=1.6.0 transformers>=4.11.3 sentencepiece

In [11]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=157ef157c4dc5ae16bec4d7f9b47d9d1626bfc4b1dd9f580ac52c89771528042
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [16]:
import nlpaug.augmenter.word as naw
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en',
    device='cuda'
)
augmented_text = back_translation_aug.augment('John is going to town')
augmented_text

['John goes to town']

In [18]:
from tqdm import tqdm

In [19]:
aug = []

for i, review in enumerate(tqdm(neg)):
  aug.append(back_translation_aug.augment(review))

100%|██████████| 224/224 [11:03<00:00,  2.96s/it]


In [25]:
augmented_neg = []

for lst in aug:
  augmented_neg.append(lst[0])

In [27]:
augmented_label = [0] * len(augmented_neg)

In [32]:
augmented_data = pd.DataFrame([augmented_neg, augmented_label]).T

In [34]:
augmented_data.columns = ['TEXT', 'sentiment']

In [42]:
negdata = pd.concat([augmented_data, negtrue], ignore_index=True)

In [43]:
negdata

,TEXT,sentiment
0,I was in possession of this car for 6 months. ...,0
1,We bought this vehicle in Jan after months of ...,0
2,I bought the Kia Optima a year ago - this is m...,0
3,"I have a 2007 Nissan versa, I have startup pro...",0
4,I was so excited about the redesigned 2007 Pas...,0
...,...,...
443,"It makes loud brake noises, especially at low ...",0
444,I had a check engine the 2nd. week of purchase...,0
445,This car was a big disappointment after drivin...,0
446,Bought this car new. Driven mostly highway mil...,0


In [45]:
posdata = dataset[dataset['sentiment'] == 1]
posdata = posdata.sample(552)

In [46]:
balanceddata = pd.concat([negdata, posdata], ignore_index=True)

In [48]:
balanceddata

,TEXT,sentiment
0,I was in possession of this car for 6 months. ...,0
1,We bought this vehicle in Jan after months of ...,0
2,I bought the Kia Optima a year ago - this is m...,0
3,"I have a 2007 Nissan versa, I have startup pro...",0
4,I was so excited about the redesigned 2007 Pas...,0
...,...,...
995,"I just graduated high school, and was looking ...",1
996,"Wherever you go, you can hear people announce ...",1
997,"I have owned a 2005 Maxima, which was totaled ...",1
998,This car has exceeded my high expectations. Pe...,1


In [50]:
balanceddata.to_csv('balanced_labels_posneg.csv')